In [ ]:
import os
import re
import requests

word = input("请输入要搜索的图片：")
if not os.path.exists('c:/Temp/' + word):
    os.mkdir('c:/Temp/' + word)

url = "https://image.baidu.com/search/index?tn=baiduimage&ipn=r&ct=201326592\
&cl=2&lm=-1&st=-1&fm=result&fr=&sf=1&fmq=1540822430688_R&pv=&ic=0&nc=1&z=&se=1\
&showtab=0&fb=0&width=&height=&face=0&istype=2&ie=utf-8&word=" + word

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}

r = requests.get(url, headers=header, timeout=3).content.decode("utf-8")

img_list = re.findall(r'"objURL":"(.*?)",', r)

for img in img_list:
    print(img)
    end = re.search(r'(.jpg|.png|.gif|.jpeg)$', img)
    if end == None:
        img = img +'.jpg'

    path = re.sub('\?|\/', '', img[-10:])

    try:
        with open('c:/Temp/' + word +'/{}'.format(path), 'ab') as f:
            ret = requests.get(img, headers=header, timeout=3)
            f.write(ret.content)
    except Exception:
        pass

In [ ]:
from _md5 import md5
import requests
from selenium import webdriver
import re
from multiprocessing import Pool

# 获取完整网页源码 
def get_one_html(url): 
    browser = webdriver.Phantomjs()
    browser.get(url) 
    p_source = browser.page_source
    browser.close() 
    return p_source 

# 用正则表达式提取链接并且返回URL 
def parse_html(html): 
    pattern = re.compile(r'<li.*?<img src="(.*?)".*?/>.*?</li>', re.S)
    urls = re.findall(pattern, html)
    print(urls)
    return urls

# 下载图片，注意的是文件写入的是二进制，并且文件名使用MD5生成固定长度的十六进制，防止文件名冲突，index表示页数 
def download_imgs(urls):
    print('one page')
    for index, url in enumerate(urls): 
        response = requests.get(url) 
        content = response.content
        file_path = '{0}\\{1}{2}{3}'.format('E:\\jdpics', md5(content).hexdigest(), index, '.jpg')
        with open(file_path, 'wb') as f: 
            f.write(content) 
            print('当前页面爬取完成')

# 构造一个可以变化的URL，并下载
def main(i): 
    url = 'http://jandan.net/ooxx/page-'+str(i)+'#comments' 
    html = get_one_html(url) 
    urls = parse_html(html)
    download_imgs(urls) 
    
# 实现一个多进程爬虫 
if __name__ == '__main__':
    # for i in range(1, 85):
    #     url = 'http://jandan.net/ooxx/page-'+str(i)+'#comments'
    #     main(url)
    pool = Pool()
    pool.map(main, [i for i in range(1, 85)])

In [ ]:
import os
import re
import requests
from selenium import webdriver
import time

word = "桌面"

url = "https://image.baidu.com/search/index?tn=baiduimage&ipn=r&ct=201326592\
&cl=2&lm=-1&st=-1&fm=result&fr=&sf=1&fmq=1540822430688_R&pv=&ic=0&nc=1&z=&se=1\
&showtab=0&fb=0&width=&height=&face=0&istype=2&ie=utf-8&word=" + word

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}

browser = webdriver.Chrome() # 或者使用Firefox驱动 webdriver.Firefox()
browser.get(url)
browser.implicitly_wait(10)

i = 0
for i in range(5):# 设置下拉5次
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')# 下拉到最底部
    time.sleep(5)# 防止网络延迟，卡顿等

r = browser.page_source
browser.close()
print(r)

img_list = re.findall(r'"objURL":"(.*?)",', r)
print(img_list)

for img in img_list:
    print(img)

'''
    path = re.sub('\?|\/', '', img[-10:])

    try:
        with open('c:/Temp/' + word +'/{}'.format(path), 'ab') as f:
            ret = requests.get(img, headers=header, timeout=30)
            f.write(ret.content)
    except Exception:
        pass
'''

In [ ]:
#查询中标公告 旧版网站

import requests
import re
from lxml import etree

class ZbSpider(object):
    def __init__(self):
        self.url = "http://www.ccgp-qinghai.gov.cn/jilin/zbxxController.form?"
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}
        self.page = 0
        
    def start_request(self):
        lx = input("请输入公告类型（1.省级 2.州市县级）：")
        pg = input("请输入爬取页数：")
        for self.page in range(int(pg)):
            page = self.page + 1
            print("正在爬取中标公告第【%d】页...-----------------------------" % page)
            html = requests.get(self.url+"declarationType=W"+"&type="+lx+"&pageNo="+
str(self.page), headers=self.headers).content.decode()
            self.zbxx(html)
        '''
        # 废流标公告信息
        self.page = 0
        for self.page in range(int(pg)):
            print("正在爬取废流标公告第【%d】页...---------------------------" % self.page)
            html = requests.get(self.url+"declarationType=F"+"&type="+lx+"&pageNo="+
str(self.page), headers=self.headers).content.decode()
            self.zbxx(html)
        '''
    
    #中标公告信息获取
    def zbxx(self, html):
        #stitle = re.findall(r'title="(.*?)"', html)    #公告标题
        surl = re.findall(r'" href="(.*?)">', html)
        for su in surl:
            #实际公告页面在框架内，去除框架页多余字符，合成实际url
            sggurl = su.replace('ftl/jilin/noticeDetail.jsp?htmlURL=','')
            sgg = requests.get(sggurl, headers=self.headers).content
            s = etree.HTML(sgg)
            name = s.xpath('/html/body/div/table/tbody/tr[2]/td[2]/p/span/text()')
            date = s.xpath('/html/body/div/table/tbody/tr[8]/td[2]/p/span/text()')
            man = s.xpath('/html/body/div/table/tbody/tr[13]/td[2]/p/span/text()')
            print('【'+date[0]+'】',name[0],'【'+man[0]+'】')
            print(sggurl)    #打印公告链接
if __name__ == "__main__":
    zb = ZbSpider()
    zb.start_request()

In [ ]:
#查询中标公告 V1.0

import requests
import re
import json
import html

class ZbSpider(object):
    
    url = "http://www.ccgp-qinghai.gov.cn/es-articles/es-article/_search"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0", "Content-Type": "application/json"} # 提交查询类型为application/json
    # Payload查询参数json
    payloadData = {
        "from": 0,
        "size": 1,
        "query": {
            "bool": {
                "must": [
                    {
                        "term": {
                            "siteId": {
                                "value": "38",
                                "boost": 1
                            }
                        }
                    },
                    {
                        "wildcard": {
                            "path": {
                                "wildcard": "*6zcyannouncement46*",
                                "boost": 1
                            }
                        }
                    }
                ],
                "adjust_pure_negative": True,
                "boost": 1,
                "should": []
            }
        },
        "sort": [
            {
                "publishDate": {
                    "order": "desc"
                }
            },
            {
                "_id": {
                    "order": "desc"
                }
            }
        ],
        "_source": {
            "includes": [
                "title",
                "articleId",
                "siteId",
                "cover",
                "url",
                "pathName",
                "publishDate",
                "attachmentUrl",
                "districtName",
                "gpCatalogName"
            ],
            "excludes": [
                "content"
            ]
        }
    }        
    zbgg_addrs = []
    
    def __init__(self, dfrom, dsize):
        self.payloadData['from'] = dfrom # 中标公告列表起始位置
        self.payloadData['size'] = dsize # 总数
 
    # 获取中标公告列表   
    def zbgg_list(self): 
        html_text = requests.post(self.url, data=json.dumps(self.payloadData), headers=self.headers)
        #print(html_text)
        d = json.loads(html_text.content.decode())
        size = self.payloadData['size']
        for i in range(size):
            self.zbgg_addrs.append("http://www.ccgp-qinghai.gov.cn"+d[ 'hits']['hits'][i]['_source']['url']) # 获取中标公告页面地址
            # d[ 'hits']['hits'][i]['_source']['title'] 中标公告标题，
            # d[ 'hits']['hits'][i]['_source']['url'] 中标公告页面路径

    # 打印中标公告信息    
    def zbgg_print(self): 
        i = 1
        for zbgg_adddr in self.zbgg_addrs:
            html_text = requests.get(zbgg_adddr, headers=self.headers, timeout=50).content.decode()
            html_text = html.unescape(html_text) # 反转义
            #pbzj = []
            #pbzj = re.findall(r'code-85005\\">(.*?)<', html_text) # 匹配评审专家名单
            #if pbzj == [] :
            pbzj = re.findall(r'名单：(.*?)十', html_text) # 匹配不规范页面中的评审专家名单
            pattern = re.compile(u"[\u4e00-\u9fa5、（）]") # 提取专家名单，保留汉字和"、（）"等符号，去除多余字符
            pbzj[0] = "".join(pattern.findall(pbzj[0])) # 连接成专家名单字符串
            cjrq = re.findall(r'code-94002\\">(.*?)<', html_text) # 匹配成交日期
            xmmc = re.findall(r'code-00003\\">(.*?)<', html_text) # 匹配项目名称
            xmbh = re.findall(r'code-00004\\">(.*?)<', html_text) # 匹配项目编号
            print("["+str(i)+"] "+cjrq[0]+", "+xmbh[0]+", "+xmmc[0]+", "+pbzj[0])
            i = i + 1
    
if __name__ == "__main__":
    zb = ZbSpider(0,100)
    zb.zbgg_list()
    zb.zbgg_print()    

In [1]:
#查询中标公告 V2.0

import requests
import re
import json
import html

class ZbSpider(object):
    
    url = "http://www.ccgp-qinghai.gov.cn/front/search/category"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0", "Content-Type": "application/json"} # 提交查询类型为application/json
    # Payload查询参数json
    payloadData = {
        "pageNo":1,
        "pageSize":1,
        "categoryCode":"ZcyAnnouncement4",
        "districtCode":["639900"]
    }        
    zbgg_addrs = []
    
    def __init__(self, dfrom, dsize):
        self.payloadData['pageNo'] = dfrom # 中标公告列表起始位置
        self.payloadData['pageSize'] = dsize # 总数
 
    # 获取中标公告列表   
    def zbgg_list(self): 
        html_text = requests.post(self.url, data=json.dumps(self.payloadData), headers=self.headers)
        d = json.loads(html_text.content.decode())
        size = self.payloadData['pageSize']
        for i in range(size):
            self.zbgg_addrs.append("http://www.ccgp-qinghai.gov.cn"+d[ 'hits']['hits'][i]['_source']['url']) # 获取中标公告页面地址
            # d[ 'hits']['hits'][i]['_source']['title'] 中标公告标题，
            # d[ 'hits']['hits'][i]['_source']['url'] 中标公告页面路径

    # 打印中标公告信息    
    def zbgg_print(self): 
        i = 1
        for zbgg_adddr in self.zbgg_addrs:
            html_text = requests.get(zbgg_adddr, headers=self.headers, timeout=50).content.decode()
            html_text = html.unescape(html_text) # 反转义
            #pbzj = re.findall(r'code-85005\\">(.*?)<', html_text) # 匹配评审专家名单
            pbzj = re.findall(r'名单：(.*?)十', html_text) # 匹配不规范页面中的评审专家名单
            pattern = re.compile(u"[\u4e00-\u9fa5、（）]") # 提取专家名单，保留汉字和"、（）"等符号，去除多余字符
            pbzj[0] = "".join(pattern.findall(pbzj[0])) # 连接成专家名单字符串
            cjrq = re.findall(r'code-94002 date-selection-cls\\">(.*?)<', html_text) # 匹配成交日期
            xmmc = re.findall(r'code-00003 single-line-text-input-box-cls\\">(.*?)<', html_text) # 匹配项目名称
            xmbh = re.findall(r'code-00004 single-line-text-input-box-cls\\">(.*?)<', html_text) # 匹配项目编号
            print("["+str(i)+"] "+cjrq[0]+", "+xmbh[0]+", "+xmmc[0]+", "+pbzj[0])
            i = i + 1
    
if __name__ == "__main__":
    zb = ZbSpider(1,50)
    zb.zbgg_list()
    zb.zbgg_print()    

[1] 2020-03-13, 青海诚德公招（货物）2020-005, 2020年普通动物疫病疫苗采购项目, 杜银忠、师玉宝、陆艳、陈雷、王生祥、姜立红、段海军
[2] 2020-03-12, 青海诚德公招（货物）2020-004, 2020年全省强制免疫疫苗采购项目, 贺晓龙、李建平、沈艳丽、王生祥、赵廷贵、姜立红、段海军
[3] 2020-03-13, 青海省招公招（货物）2020-002,  甘德县民政局2020年春季疫情期间救灾粮食政府公开招标采购项目, 沈秀英（组长）、赵海霞、芦平、魏学庆、周鹏（招标人代表）
[4] 2020-03-13, 青海汇海源磋商（服务）2020-001, 城西区食品药品安全监管平台升级改造项目, 马永芳（组长）、韩青秀、张峰
[5] 2020-03-10, 青海诚鑫竞磋（服务）2020-007, 三江源民族中学二期建设项目校园景观、变配电工程监理, 韦金元、李平、吴威海
[6] 2020-03-10, 青海创鑫竞磋（货物）2020-002,  四川喇荣五明佛学院玉树籍尼姑（觉姆）囊谦集中安置点食堂设备采购项目, 郑斌（组长）、李江春、杨涛（采购人代表）
[7] 2020-03-10, 川招青海磋商（工程）2020-008, 青海省疾病预防控制中心污水处理站改造项目, 唐渝青（组长）、闫玉霞、闫刚（采购人代表）
[8] 2020-03-06, 青海诚德竞磋（服务）2020-001, “金审三期”建设项目监理服务采购, 自行选定专家洪恺、刘睿采购人代表王东立
[9] 2020-03-03, 青海国焱竞磋（服务）2020-010号, 青海省妇幼保健院迁址新建项目工程结算审计采购项目, 李宁（组长）、唐渝青和王宗海
[10] 2020-03-02, 青海省招竞磋（货物）2020-003, 某部队通信设备及卫生物资采购项目（包三）, 赵宏（组长）、秦振虎、成大华
[11] 2020-03-02, 青海省招竞磋（货物）2020-003, 某部队通信设备及卫生物资采购项目（包一）, 赵宏（组长）、秦振虎、成大华
[12] 2020-01-22, 青海百鑫公招（货物）2019-138, 青海省第五人民医院功能科多普勒超声波诊断仪采购项目（第四次）, 刘海龙、余兰元、宋国胜、李广斌、谢录玲（采购人代表）
[13] 2020-01-21, 青海闫建

In [ ]:
# _*_ coding: utf-8 _*_
# 学习强国
from selenium import webdriver
import time

__author__ = 'Silent_Coder'
__date__ = '2019/3/12 22:41'

HOME_PAGE = 'https://www.xuexi.cn/'
VIDEO_LINK = 'https://www.xuexi.cn/a191dbc3067d516c3e2e17e2e08953d6/b87d700beee2c44826a9202c75d18c85.html?pageNumber=39'
LONG_VIDEO_LINK = 'https://www.xuexi.cn/f65dae4a57fe21fcc36f3506d660891c/b2e5aa79be613aed1f01d261c4a2ae17.html'
LONG_VIDEO_LINK2 = 'https://www.xuexi.cn/0040db2a403b0b9303a68b9ae5a4cca0/b2e5aa79be613aed1f01d261c4a2ae17.html'
TEST_VIDEO_LINK = 'https://www.xuexi.cn/8e35a343fca20ee32c79d67e35dfca90/7f9f27c65e84e71e1b7189b7132b4710.html'
SCORES_LINK = 'https://pc.xuexi.cn/points/my-points.html'
LOGIN_LINK = 'https://pc.xuexi.cn/points/login.html'
ARTICLES_LINK = 'https://www.xuexi.cn/d05cad69216e688d304bb91ef3aac4c6/9a3668c13f6e303932b5e0e100fc248b.html'

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
browser = webdriver.Chrome(options=options)


def login_simulation():
    """模拟登录"""
    # 方式一：使用cookies方式
    # 先自己登录，然后复制token值覆盖
    # cookies = {'name': 'token', 'value': ''}
    # browser.add_cookie(cookies)

    # 方式二：自己扫码登录
    browser.get(LOGIN_LINK)
    browser.maximize_window()
    browser.execute_script("var q=document.documentElement.scrollTop=1000")
    time.sleep(10)
    browser.get(HOME_PAGE)
    print("模拟登录完毕\n")


def watch_videos():
    """观看视频"""
    browser.get(VIDEO_LINK)
    videos = browser.find_elements_by_xpath("//div[@id='Ck3ln2wlyg3k00']")
    spend_time = 0

    for i, video in enumerate(videos):
        if i > 6:
            break
        video.click()
        all_handles = browser.window_handles
        browser.switch_to_window(all_handles[-1])
        browser.get(browser.current_url)

        # 点击播放
        browser.find_element_by_xpath("//div[@class='outter']").click()
        # 获取视频时长
        video_duration_str = browser.find_element_by_xpath("//span[@class='duration']").get_attribute('innerText')
        video_duration = int(video_duration_str.split(':')[0]) * 60 + int(video_duration_str.split(':')[1])
        # 保持学习，直到视频结束
        time.sleep(video_duration + 3)
        spend_time += video_duration + 3
        browser.close()
        browser.switch_to_window(all_handles[0])

    # if spend_time < 3010:
    #     browser.get(LONG_VIDEO_LINK)
    #     browser.execute_script("var q=document.documentElement.scrollTop=850")
    #     try:
    #         browser.find_element_by_xpath("//div[@class='outter']").click()
    #     except:
    #         pass
    #
    #     # 观看剩下的时间
    #     time.sleep(3010 - spend_time)
    browser.get(TEST_VIDEO_LINK)
    time.sleep(3010 - spend_time)
    print("播放视频完毕\n")


def read_articles():
    """阅读文章"""
    browser.get(ARTICLES_LINK)
    articles = browser.find_elements_by_xpath("//div[@id='Ca4gvo4bwg7400']")
    for index, article in enumerate(articles):
        if index > 7:
            break
        article.click()
        all_handles = browser.window_handles
        browser.switch_to_window(all_handles[-1])
        browser.get(browser.current_url)
        for i in range(0, 2000, 100):

            js_code = "var q=document.documentElement.scrollTop=" + str(i)
            browser.execute_script(js_code)
            time.sleep(5)
        for i in range(2000, 0, -100):
            js_code = "var q=document.documentElement.scrollTop=" + str(i)
            browser.execute_script(js_code)
            time.sleep(5)
        time.sleep(80)
        browser.close()
        browser.switch_to_window(all_handles[0])
    print("阅读文章完毕\n")


def get_scores():
    """获取当前积分"""
    browser.get(SCORES_LINK)
    time.sleep(2)
    gross_score = browser.find_element_by_xpath("//*[@id='app']/div/div[2]/div/div[2]/div[2]/span[1]")\
        .get_attribute('innerText')
    today_score = browser.find_element_by_xpath("//span[@class='my-points-points']").get_attribute('innerText')
    print("当前总积分：" + str(gross_score))
    print("今日积分：" + str(today_score))
    print("获取积分完毕，即将退出\n")


if __name__ == '__main__':
    login_simulation()  # 模拟登录
    read_articles()     # 阅读文章
    watch_videos()      # 观看视频
    get_scores()        # 获得今日积分
    browser.quit()